In [1]:
import torch
import numpy as np
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import random
import autograd_lib as agl
from torch.autograd.functional import hessian
from torch.autograd import Variable
#from functorch import hessian

In [2]:
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
x = np.expand_dims(np.arange(-1.5, 1.5, 0.01),1) 
print(x.shape)
y = np.sinc(5*x)
print(y.shape)

(300, 1)
(300, 1)


In [7]:
x, y = torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [61]:
class Model1(nn.Module):
    def __init__(self,):
        super(Model1, self).__init__()
        self.linear1 = nn.Linear(1, 5)
        self.linear2 = nn.Linear(5, 10)
        self.linear3 = nn.Linear(10, 10)
        self.linear4 = nn.Linear(10, 10)
        self.linear5 = nn.Linear(10, 10)
        self.linear6 = nn.Linear(10, 10)
        self.linear7 = nn.Linear(10, 5)
        self.predict = nn.Linear(5, 1)
    
    def forward(self,x):
        x = nn.functional.leaky_relu(self.linear1(x))
        x = nn.functional.leaky_relu(self.linear2(x))
        x = nn.functional.leaky_relu(self.linear3(x))
        x = nn.functional.leaky_relu(self.linear4(x))
        x = nn.functional.leaky_relu(self.linear5(x))
        x = nn.functional.leaky_relu(self.linear6(x))
        x = nn.functional.leaky_relu(self.linear7(x))

        x = self.predict(x)
        return x
 
model_1 = Model1()

In [86]:
optimizer = torch.optim.RMSprop(model_1.parameters(), lr = 1e-3, weight_decay = 1e-4)
loss_func = torch.nn.MSELoss()  # mean squared loss
pytorch_total_params = sum(p.numel() for p in model_1.parameters())
minimal_ratio_list=[]
train_losslist1=[]
train_loss, minimal_ratio= model_train(model_1, x, y)

minimal_ratio_list.append(minimal_ratio)
train_losslist1.append(train_loss)

debug
debug
epoch: 1, loss = 0.0617
tensor([-9.3037e-06, -1.5305e-05,  2.5873e-05,  2.8521e-05, -1.0658e-05,
         8.8620e-06,  9.3955e-07, -9.4831e-06, -1.0660e-05, -1.1944e-05,
         4.4630e-06,  1.8716e-07,  6.2281e-06,  3.1382e-06,  9.5444e-08,
        -4.3666e-05, -4.4255e-05, -5.6151e-05, -2.4388e-05, -3.3672e-05,
         1.1595e-07,  1.1648e-07,  1.4996e-07,  6.4990e-08,  8.9715e-08,
         8.7257e-06, -2.2084e-05,  1.1140e-05,  8.2418e-06, -1.6493e-05,
         4.8853e-06,  8.1973e-08,  6.5230e-06,  3.3561e-06,  7.0975e-10,
        -2.3882e-07, -2.2307e-07, -3.0756e-07, -1.3550e-07, -1.7053e-07,
         5.5956e-08,  8.6435e-08,  7.8791e-08,  2.8892e-08,  7.4277e-08,
        -7.2276e-09, -2.2467e-09, -1.0361e-08, -4.7457e-09, -3.0632e-09,
        -1.3792e-07, -1.9453e-07, -1.8851e-07, -7.2499e-08, -1.6184e-07,
        -1.5421e-07, -1.7224e-07, -1.9941e-07, -8.4493e-08, -1.3247e-07,
         6.8176e-06, -8.7374e-05,  2.3152e-07,  5.4136e-06,  7.6301e-06,
        -4.7037

AttributeError: 'NoneType' object has no attribute 'flatten'

In [29]:
num_param = sum(p.numel() for p in model_1.parameters())
print(num_param)
names = list(n for n, _ in model_1.named_parameters())

571


In [84]:
def model_train(model, x, y):
    print("debug")
    loss_arr, minimal_list=[],[]
    n_epochs = [*range(10)]
    for epoch in range(1, len(n_epochs)+1):
        print("debug")
        prediction = model(x)  
        loss = loss_func(prediction, y)
        optimizer.zero_grad() #clear gradients for next epoch
        loss.backward() #compute gradients using back propogation
        optimizer.step() 
        loss.requires_grad_()
        print(f'epoch: {epoch}, loss = {loss.item():.4f}')
        #J = torch.autograd.grad(loss, list(model.parameters()), retain_graph=True)
        #J = torch.cat([e.flatten() for e in J]) # flatte
        J = torch.cat([p.grad.flatten() for p in model.parameters()]) # flatten
        print(J)
        J.requires_grad_()
        
        grad_all=0.0
        for p in model.parameters():
            #print('debug3')
            grad=0.0
            if p.grad is not None:
                #print('debug1')
                grad= (p.grad.cpu().data.numpy()**2).sum()
            grad_all+=grad
        grad_norm=grad_all**0.5
        #print('debug2')
        print(grad_norm)
        
        if(grad_norm<3.50):
            # Fill in Hessian
            model.zero_grad()
            H= torch.zeros((num_param, num_param))
            for i in range(num_param):
                #J[i].backward(retain_graph=True)
                #H[i] = torch.cat([p.grad.flatten() for p in J[i]])
                result = torch.autograd.grad(J[i], list(model.parameters()), retain_graph=True,allow_unused=True)
                print(result)
                H[i] = torch.cat([r.flatten() for r in result])
                
            eigenvalues, _ = torch.symeig(H, eigenvectors=False)
            print(eigenvalues)
            minimal_ratio = eigenvalues.min() / eigenvalues.sum()
            print(minimal_ratio)
            minimal_list.append(minimal_ratio)
            loss_arr.append(loss.detach().numpy())
            break
        

                
    return loss_arr, minimal_list

In [ ]:
train_losslist1 = []
minimal_ratio_list=[]
valid_loss_min = np.Inf
for i in range(1):
    n_epochs = [*range(10)]
    valid_loss_min = np.Inf 
    plt.figure()
    for epoch in range(1, len(n_epochs)+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model1.train()
        for data, target in train_loader:
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model1(data)
            loss = criterion(output, target)
            loss.backward()
            loss.requires_grad_()
            # Calculate Jacobian w.r.t. model parameters
            
            optimizer.step()
            train_loss += loss.item()*data.size(0)
        model1.eval()
        J = torch.cat([p.grad.flatten() for p in model1.parameters()]) # flatten
        print(num_param)
        print(J)
        J.requires_grad_()
        train_loss = train_loss/len(train_loader.dataset)
   
        grad_all=0.0
        for p in model1.parameters():
            #print('debug3')
            grad=0.0
            if p.grad is not None:
                #print('debug1')
                grad= (p.grad.cpu().data.numpy()**2).sum()
            grad_all+=grad
        grad_norm=grad_all**0.5
        #print('debug2')
        print(grad_norm)
        #plt.scatter(epoch,grad_norm)
        # ---------------------------------minimal ratio-------------------------------------------- 
        if(grad_norm<3.50):
            # Fill in Hessian
            model1.zero_grad()
            H= torch.zeros((num_param, num_param))
            for i in range(num_param):
                J[i].backward(retain_graph=True)
                H[i] = torch.cat([p.grad.flatten() for p in model1.parameters()])
                
            eigenvalues, _ = torch.symeig(H, eigenvectors=False)
            minimal_ratio = eigenvalues.min() / eigenvalues.sum()
            minimal_ratio_list.append(minimal_ratio)
            train_losslist1.append(train_loss)
            break
    
    
    # ---------------------------------minimal ratio------------------------------------------------ 




        
        for data, target in valid_loader:
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            output = model1(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
        
        #train_loss = train_loss/len(train_loader.dataset)
        valid_loss = valid_loss/len(valid_loader.dataset)
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
            torch.save(model1.state_dict(), 'model_cifar.pt')
            valid_loss_min = valid_loss
        
plt.scatter(minimal_ratio_list, train_losslist1)
plt.xlabel("minimal ratio")
plt.ylabel("Loss")
plt.title("minimal ration vs loss")
plt.show()        